In [8]:
# ! pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python

In [9]:
### LLM

local_llm = 'llama3'
# local_llm = 'phi-3'

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.output_parsers import JsonOutputParser


from typing_extensions import TypedDict
from typing import List
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

import os


In [11]:
#Load pdfs from source
# docs = DirectoryLoader("docs/")
# data = docs.load()

In [12]:
#Load pdfs from source
docs = PyPDFDirectoryLoader("docs/")
data = docs.load()

In [13]:
data

[Document(page_content='AYUNTAMIENTO DE SANTANDER\nResolución de adjudicación del contrato para el servicio\nde vigilancia en varias dependencias municipales, lotes\n1, 2 y 3.\n1.Entidad adjudicadora:Ayuntamiento de Santander,\nServicio de Contratación.\nNúmero de expediente:184/98.2.Objeto del contrato:Servicio de vigilancia en varias\ndependencias:\nLote 1:Conservatorio Municipal.Lote 2:Camping Municipal.Lote 3:Edificios SEMAS (Tojo y Pronillo).Boletín y fecha de publicación del anuncio de licita-\nción:BOC número 246, de 10 de diciembre de 1998.\n3. Tramitación anticipada: Procedimiento abierto y\nforma de adjudicación concurso.\n4.Presupuesto de licitación:Lote 1:4.000.000 de pesetas/año.Lote 2:1.850.000 pesetas/año.Lote 3:37.500.000 pesetas/año.5.Adjudicación:Lotes 1 y 2:2-7-99.Lote 3:18-5-99.Contratista:«La Estela Cántabra de Seguridad,\nSociedad Limitada».\nNacionalidad:Española.Importe lote 1:3.628.625 pesetas/año.Lote 2:1.598.480 pesetas/año.Lote 3:34.447.824 pesetas/año.Santa

In [14]:
# from sentence_transformers import SentenceTransformer

# embedding_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

Mirar esto https://iamajithkumar.medium.com/how-to-use-chroma-to-build-your-first-similarity-search-5c054bfd5add

In [15]:


### Index
# data = PyPDFLoader("1.pdf")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(data)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=GPT4AllEmbeddings(),
)
retriever = vectorstore.as_retriever()


In [41]:
# retriever = vectorstore.as_retriever(k=2)
documents = retriever.invoke("1 de diciembre de 1999")

In [42]:
documents

[Document(page_content='99/379879BOC - Número 237 Miércoles, 1 de diciembre de 1999 Página 8297', metadata={'page': 1, 'source': 'docs\\27.pdf'}),
 Document(page_content='99/379314\n______2.2 CURSOS, OPOSICIONES Y CONCURSOS  ______\nCONSEJERÍA DE PRESIDENCIA\nOrden de 9 de noviembre de 1999, por la que se convoca\nla provisión, mediante el sistema de libre designación, de\npuestos de trabajo de la Consejería de Educación y\nJuventud, reservados a funcionarios de carrera.\nVisto lo dispuesto en el Real Decreto 364/1995, de 10', metadata={'page': 0, 'source': 'docs\\2.pdf'}),
 Document(page_content='99/379314\n______2.2 CURSOS, OPOSICIONES Y CONCURSOS  ______\nCONSEJERÍA DE PRESIDENCIA\nOrden de 9 de noviembre de 1999, por la que se convoca\nla provisión, mediante el sistema de libre designación, de\npuestos de trabajo de la Consejería de Educación y\nJuventud, reservados a funcionarios de carrera.\nVisto lo dispuesto en el Real Decreto 364/1995, de 10', metadata={'page': 0, 'source': 'd

In [34]:
### Generate

from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise . SPanish<|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "Que paso el 1 de diciembre de 1999"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Según el contexto, el 1 de diciembre de 1999 es el día en que se fija el período voluntario de recaudación de impuestos.


In [19]:
# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>Eres una asistente para respuesta de preguntas. 
    Usa el contexto proporcionado para contestar a la pregunta. Si no conoces la respuesta, di que no conoces la respuesta. 
    Responde siempre en español y con 4 frases como mucho. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Chain
rag_chain = prompt | llm | StrOutputParser()

In [20]:
### State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents 
    """
    question : str
    generation : str
    documents : List[str]



### Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """

   
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    
    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

# def grade_documents(state):
#     """
#     Determines whether the retrieved documents are relevant to the question
#     If any document is not relevant, we will set a flag to run web search

#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): Filtered out irrelevant documents and updated web_search state
#     """

#     print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
#     question = state["question"]
#     documents = state["documents"]
    
#     # Score each doc
#     filtered_docs = []
#     for d in documents:
#         score = retrieval_grader.invoke({"question": question, "document": d.page_content})
#         grade = score['score']
#         # Document relevant
#         if grade.lower() == "yes":
#             print("---GRADE: DOCUMENT RELEVANT---")
#             filtered_docs.append(d)
#         # Document not relevant
#         else:
#             print("---GRADE: DOCUMENT NOT RELEVANT---")
#             # We do not include the document in filtered_docs
#             # We set a flag to indicate that we want to run web search
#             continue
#     return {"documents": filtered_docs, "question": question}
    



In [21]:
from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve) # retrieve
# workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

# Build graph
workflow.set_entry_point("retrieve")
# workflow.add_edge("retrieve", "grade_documents")
# workflow.add_edge("grade_documents", "generate")
workflow.add_edge("retrieve", "generate")
workflow.set_finish_point("generate")

In [22]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint
inputs = {"question": """  6 de marzo de 2000"""}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---RETRIEVE---
'Finished running: retrieve:'
---GENERATE---
'Finished running: generate:'
'La fecha mencionada es el 6 de marzo de 2000.'


In [23]:
# ### Index

# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import GPT4AllEmbeddings

# urls = [
#     "https://lilianweng.github.io/posts/2023-06-23-agent/",
#     "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
#     "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
# ]

# docs = [WebBaseLoader(url).load() for url in urls]
# docs_list = [item for sublist in docs for item in sublist]

# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=250, chunk_overlap=0
# )
# doc_splits = text_splitter.split_documents(docs_list)

# # Add to vectorDB
# vectorstore = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=GPT4AllEmbeddings(),
# )
# retriever = vectorstore.as_retriever()

In [24]:
# ### Retrieval Grader 

# from langchain.prompts import PromptTemplate
# from langchain_community.chat_models import ChatOllama
# from langchain_core.output_parsers import JsonOutputParser

# # LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance 
#     of a retrieved document to a user question. If the document contains keywords related to the user question, 
#     grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
#     Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
#     Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
#      <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Here is the retrieved document: \n\n {document} \n\n
#     Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
#     """,
#     input_variables=["question", "document"],
# )

# retrieval_grader = prompt | llm | JsonOutputParser()
# question = "Molledo"
# docs = retriever.invoke(question)
# doc_txt = docs[1].page_content
# print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

In [25]:
# ### Generate



# # Prompt
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
#     Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
#     Use three sentences maximum and keep the answer concise. <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Question: {question} 
#     Context: {context} 
#     Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question", "document"],
# )

# llm = ChatOllama(model=local_llm, temperature=0)

# # Post-processing
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# # Chain
# rag_chain = prompt | llm | StrOutputParser()

# # Run
# question = "Que sabes del ayuntamiento de Molledo"
# docs = retriever.invoke(question)
# generation = rag_chain.invoke({"context": docs, "question": question})
# print(generation)

In [26]:
# ### Hallucination Grader 

# # LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# # Prompt
# prompt = PromptTemplate(
#     template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether 
#     an answer is grounded in / supported by a set of facts. Give a binary score 'yes' or 'no' score to indicate 
#     whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
#     single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Here are the facts:
#     \n ------- \n
#     {documents} 
#     \n ------- \n
#     Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["generation", "documents"],
# )

# hallucination_grader = prompt | llm | JsonOutputParser()
# hallucination_grader.invoke({"documents": docs, "generation": generation})

In [27]:
# ### Answer Grader 

# # LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# # Prompt
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an 
#     answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
#     useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
#      <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
#     \n ------- \n
#     {generation} 
#     \n ------- \n
#     Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["generation", "question"],
# )

# answer_grader = prompt | llm | JsonOutputParser()
# answer_grader.invoke({"question": question,"generation": generation})

In [28]:
# ### Router

# from langchain.prompts import PromptTemplate
# from langchain_community.chat_models import ChatOllama
# from langchain_core.output_parsers import JsonOutputParser

# # LLM
# llm = ChatOllama(model=local_llm, format="json", temperature=0)

# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
#     user question to a vectorstore or web search. Use the vectorstore for questions on LLM  agents, 
#     prompt engineering, and adversarial attacks. You do not need to be stringent with the keywords 
#     in the question related to these topics. Otherwise, use web-search. Give a choice 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
#     no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question"],
# )

# question_router = prompt | llm | JsonOutputParser()
# question = "Molledo"
# docs = retriever.get_relevant_documents(question)
# doc_txt = docs[1].page_content
# print(question_router.invoke({"question": question}))

We'll implement these as a control flow in LangGraph.

### Graph Build

In [29]:
# Compile
app = workflow.compile()

# Test
from pprint import pprint
inputs = {"question": "Que sabes del ayuntamiento de Santoña"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---RETRIEVE---
'Finished running: retrieve:'
---GENERATE---
'Finished running: generate:'
('Sé que el Ayuntamiento de Santoña es una entidad municipal española que ha '
 'adoptado un acuerdo para modificar la tasa por prestación del servicio de '
 'recogida de basura en noviembre de 1999.')


Trace: 

https://smith.langchain.com/public/8d449b67-6bc4-4ecf-9153-759cd21df24f/r

Trace: 

https://smith.langchain.com/public/c785f9c0-f519-4a38-ad5a-febb59a2139c/r

In [30]:
# def web_search(state):
#     """
#     Web search based based on the question

#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): Appended web results to documents
#     """

#     print("---WEB SEARCH---")
#     question = state["question"]
#     documents = state["documents"]

#     # Web search
#     docs = web_search_tool.invoke({"query": question})
#     web_results = "\n".join([d["content"] for d in docs])
#     web_results = Document(page_content=web_results)
#     if documents is not None:
#         documents.append(web_results)
#     else:
#         documents = [web_results]
#     return {"documents": documents, "question": question}

# ### Conditional edge

# def route_question(state):
#     """
#     Route question to web search or RAG.

#     Args:
#         state (dict): The current graph state

#     Returns:
#         str: Next node to call
#     """

#     print("---ROUTE QUESTION---")
#     question = state["question"]
#     print(question)
#     source = question_router.invoke({"question": question})  
#     print(source)
#     print(source['datasource'])
#     if source['datasource'] == 'web_search':
#         print("---ROUTE QUESTION TO WEB SEARCH---")
#         return "websearch"
#     elif source['datasource'] == 'vectorstore':
#         print("---ROUTE QUESTION TO RAG---")
#         return "vectorstore"

# def decide_to_generate(state):
#     """
#     Determines whether to generate an answer, or add web search

#     Args:
#         state (dict): The current graph state

#     Returns:
#         str: Binary decision for next node to call
#     """

#     print("---ASSESS GRADED DOCUMENTS---")
#     question = state["question"]
#     web_search = state["web_search"]
#     filtered_documents = state["documents"]

#     if web_search == "Yes":
#         # All documents have been filtered check_relevance
#         # We will re-generate a new query
#         print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
#         return "websearch"
#     else:
#         # We have relevant documents, so generate answer
#         print("---DECISION: GENERATE---")
#         return "generate"

# ### Conditional edge

# def grade_generation_v_documents_and_question(state):
#     """
#     Determines whether the generation is grounded in the document and answers question.

#     Args:
#         state (dict): The current graph state

#     Returns:
#         str: Decision for next node to call
#     """

#     print("---CHECK HALLUCINATIONS---")
#     question = state["question"]
#     documents = state["documents"]
#     generation = state["generation"]

#     score = hallucination_grader.invoke({"documents": documents, "generation": generation})
#     grade = score['score']

#     # Check hallucination
#     if grade == "yes":
#         print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
#         # Check question-answering
#         print("---GRADE GENERATION vs QUESTION---")
#         score = answer_grader.invoke({"question": question,"generation": generation})
#         grade = score['score']
#         if grade == "yes":
#             print("---DECISION: GENERATION ADDRESSES QUESTION---")
#             return "useful"
#         else:
#             print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
#             return "not useful"
#     else:
#         pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
#         return "not supported"
